In [1]:
import torch
import torch.nn as nn
import os
from ode_nn import AutoODE_COVID, weight_fun
from ode_nn import Dataset, train_epoch, eval_epoch, get_lr
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils import data
import matplotlib.pyplot as plt
import random
import warnings
from ode_nn import Dataset_graph, train_epoch_graph, eval_epoch_graph, get_lr
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using backend: pytorch
/global/homes/r/rwang2/.conda/envs/myenv/lib/python3.6/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)


## Data Preprocessing

In [12]:
# Read and Preprocess the csv files from John Hopkins Dataset
# https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us
direc = ".../ODEs/Data/COVID/" # Directory that contains daily report csv files.
list_csv = sorted(os.listdir(direc))
us = []
for file in list_csv:
    sample = pd.read_csv(direc + file).set_index("Province_State")[["Confirmed", "Recovered", "Deaths"]].sort_values(by = "Confirmed", ascending = False)
    us.append(sample.drop(['Diamond Princess', 'Grand Princess']))
us = pd.concat(us, axis=1, join='inner')
us_data = us.values.reshape(56,-1,3)
us_data[us_data!=us_data] = 0

#####################################################################################
# Normalize by total population of each state
population = pd.read_csv(".../ode_nn/population_states.csv", index_col=0)
scaler = population.loc[us.index].values.reshape(56, 1, 1)*1e6
us_data = us_data/scaler
us_data = torch.from_numpy(us_data).float().to(device)

# Mobility Data: beta = 1 - stay_at_home_percentages
beta = torch.load(".../ode_nn/mobility/us_beta.pt").float()

# U.S states 1-0 Adjacency Matrix
graph = torch.load(".../ode_nn/mobility/us_graph.pt").float()

## Train AutoODE-COVID

In [ ]:
##################################################################
test_idx = 131

# Learning Rate
lr = 0.01

# number of historic data points for fitting
input_steps = 10 

# forecasting horizon
output_steps = 7

# number of epochs for training
num_epochs = 20000

# select data for training
data = us_data[:, test_idx-input_steps:test_idx+7]
y_exact = data[:,:input_steps]

##################################################################

model = AutoODE_COVID(initial_I = data[:,0,0], initial_R = data[:,0,1], initial_D = data[:,0,2],
                      num_regions = 56, solver = "RK4", n_breaks = 1, graph = graph).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1000, gamma=0.9)
loss_fun = torch.nn.MSELoss()
min_loss = 1

##################################################################

for e in range(num_epochs):
    scheduler.step()
    y_approx = model(input_steps)
    loss = loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])
    
######## Weighted Loss ########
#     loss_weight = weight_fun(input_steps, function = "sqrt", feat_weight = True)
#     loss = torch.mean(loss_weight*loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])) 

######## A few constraints that can potential improve the model ########
#     positive_constraint = loss_fun(F.relu(-model.beta), torch.tensor(0.0).float().to(device))
#     diagonal_constraint = loss_fun(torch.diagonal(model.A, 0),torch.tensor(1.0).float().to(device))
#     initial_constraint = loss_fun(model.init_S + model.init_E + model.init_I + model.init_R + model.init_U, torch.tensor(1.0).float().to(device))
#     loss += initial_constraint + positive_constraint + diagonal_constraint 
   
    if loss.item() < min_loss:
        best_model = model
        min_loss = loss.item()
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
#     if e%1000 == 0:
#         y_approx2 = model(data.shape[1]).data.numpy()
#         y_exact2 = data.data.numpy()
#         print(list_csv[test_idx][:10])
#         #torch.mean(torch.abs(y_approx - y_exact)[:,-7:]).data, torch.mean(torch.abs(y_approx - y_exact)[:,30:]).data
#         for i in range(3):
#             print(np.mean(np.abs(y_approx2*scaler - y_exact2*scaler)[:,-7:, i]))

########################################################################
name = "autoode-covid"
y_approx = best_model(data.shape[1]).data.numpy()
y_exact = data.data.numpy()
print(list_csv[test_idx][:10])
#torch.mean(torch.abs(y_approx - y_exact)[:,-7:]).data, torch.mean(torch.abs(y_approx - y_exact)[:,30:]).data
for i in range(3):
    print(np.mean(np.abs(y_approx*scaler - y_exact*scaler)[:,-7:, i]))

torch.save({"model": best_model,
            "preds": y_approx*scaler,
            "trues": y_exact*scaler},
            ".pt")